# Testing Tools HUD

In [ ]:
from google import genai
from google.genai import types
from google.genai.types import Part
from settings.config_loader import config

client = genai.Client(api_key=config.get('api_keys.google_genai'))

weather_function = {
    "name": "get_current_temperature",
    "description": "Gets the current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. San Francisco",
            },
            
            "temperature": {
                "type": "string",
                "description": "Temperature type. 'current', 'min', or 'max'.",
                "enum": ["current", "min", "max"],
                "default": "current",
            },
            "units": {
                "type": "string",
                "description": "Units of measurement. 'metric' for Celsius, 'imperial' for Fahrenheit.",
                "enum": ["metric", "imperial"],
                "default": "metric",
            },
        },
        "required": ["location"],
    },
}

tools = types.Tool(function_declarations=[weather_function])

genai_config = types.GenerateContentConfig(
            temperature=0.4,
            system_instruction=[
                types.Part.from_text(
                    text="""
                            You are a helpful assistant. 
                            Act as a female buttler who is highly respectful and formal, also a bit of witty. 
                            Always address the user as 'Sir'. 
                            Provide accurate and concise responses.

                            You have access to the following tools to assist you:
                            1. get_calendar_events(max_results: int) -> str
                                - Fetches upcoming events or schedules from the user's Google Calendar.
                                - max_results: The maximum number of events to retrieve.
                                - Returns a formatted string of upcoming events.
                                - Response in this way :
                                    "Here are your upcoming events, Sir! Do you need any further assistance?"
                                - The events will be shown in the Frontend Display, so you don't need to include any raw data when responding.
                            2. get_current_time() -> str
                                - Retrieves the current local time.
                            3. get_date() -> str
                                - Retrieves the current date.
                            4. get_weather()
                                - Provides the current weather for a specified location.
                            When responding, if the user asks about calendar events, current time, or weather,
                                                        
                            If there is another language detected, translate and respond in english.
                    """
                )
            ],
            tools=[tools],
        )

In [15]:
response = client.models.generate_content(
                model=config.get('model.name'),
                contents=[
                    "What's the weather in San Francisco?"
                ],
                config=genai_config,
            )

In [16]:
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here:
    #  result = schedule_meeting(**function_call.args)
else:
    print("No function call found in the response.")
    print(response.text)

Function to call: get_current_temperature
Arguments: {'location': 'San Francisco'}


# Testing SearXNG

In [1]:
import requests
import logging

logger = logging.getLogger(__name__)

def search_web(query: str, max_results: int = 5) -> str:
    """Search the web using SearXNG API.
    
    Args:
        query: The search query string
        max_results: Maximum number of results to return (default: 5)
        
    Returns:
        A formatted string containing search results with titles, URLs, and snippets
    """
    try:
        logger.info(f"Searching web for: {query}")
        
        # SearXNG API endpoint
        searxng_url = "http://localhost:8888/search"
        
        # Request parameters
        params = {
            'q': query,
            'format': 'json',
            'language': 'en',
            'safesearch': 1,  # Moderate safe search
        }
        
        response = requests.get(searxng_url, params=params, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            results = data.get('results', [])
            
            if not results:
                return f"No search results found for '{query}'."
            
            # Limit results
            results = results[:max_results]
            
            # Format output
            output = f"Search Results for '{query}':\n\n"
            for i, result in enumerate(results, 1):
                title = result.get('title', 'No title')
                url = result.get('url', '')
                snippet = result.get('content', result.get('snippet', 'No description available'))
                
                # Clean up snippet (remove extra whitespace)
                snippet = ' '.join(snippet.split())
                if len(snippet) > 150:
                    snippet = snippet[:150] + '...'
                
                output += f"{i}. {title}\n"
                output += f"   {snippet}\n"
                output += f"   URL: {url}\n\n"
            
            logger.info(f"Found {len(results)} search results")
            return output.strip()
        
        elif response.status_code == 404:
            return "SearXNG service not found. Please ensure SearXNG is running at http://localhost:8888"
        else:
            return f"Search service returned error code: {response.status_code}"
            
    except requests.exceptions.ConnectionError:
        logger.error("Cannot connect to SearXNG service")
        return "Cannot connect to search service. Please ensure SearXNG is running at http://localhost:8888"
    except requests.exceptions.Timeout:
        logger.error("SearXNG search timed out")
        return "Search request timed out. Please try again."
    except Exception as e:
        logger.error(f"Error during web search: {e}", exc_info=True)
        return f"Error performing web search: {str(e)}"

In [2]:
print(search_web("Artificial Intelligence"))

Search service returned error code: 403


# Testing Search with Article Fetching

Now let's test the enhanced search that reads article content

In [ ]:
# Install beautifulsoup4 if not already installed
# pip install beautifulsoup4

from bs4 import BeautifulSoup
import requests

def fetch_article_content(url: str, max_length: int = 2000) -> str:
    """Fetch and extract main content from a URL."""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=5)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Remove script, style, and nav elements
            for tag in soup(['script', 'style', 'nav', 'header', 'footer', 'aside']):
                tag.decompose()
            
            # Try to find main content
            main_content = soup.find('article') or soup.find('main') or soup.find('div', class_='content')
            
            if main_content:
                text = main_content.get_text(separator=' ', strip=True)
            else:
                # Fallback to body
                text = soup.body.get_text(separator=' ', strip=True) if soup.body else ''
            
            # Clean up text
            text = ' '.join(text.split())
            
            # Limit length
            if len(text) > max_length:
                text = text[:max_length] + '...'
            
            return text
        else:
            return f"[Could not fetch article - Status {response.status_code}]"
            
    except Exception as e:
        return f"[Error: {e}]"

# Test fetching article
test_url = "https://techcrunch.com/category/artificial-intelligence/"
print("Fetching article content...")
content = fetch_article_content(test_url, max_length=500)
print(f"\nArticle content preview:\n{content}")